In [12]:
import os
import torch


class Para:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    data_path = 'data/cs_space_20d_rate_20d_2017_pre'

    month_num = len(os.listdir(data_path))

    model_path = 'models/model_c2_p[[0.0, 0.3], [0.7, 1.0]]_b32_lr0.001_d0.5_e500.pth'

    out_nums = 2

para = Para()
print(para.month_num)

60


In [13]:
import pandas as pd
import numpy as np
from my_utils.model_class import MLP
import torch.nn.functional as F
import rqdatac
from rqdatac import get_price

rqdatac.init()

all_factor_df = pd.DataFrame()
all_price_df = pd.DataFrame()

for i_csv in range(para.month_num):
    df = pd.read_csv(para.data_path+'/'+str(i_csv)+'.csv', index_col='order_book_id')
    df['date'] = pd.to_datetime(df['date'])

    # 加载模型
    model = MLP(in_nums=22, out_nums=para.out_nums, drop_p=0.5)
    model.load_state_dict(torch.load(para.model_path, map_location=para.device))
    model = model.to(para.device)

    # 运算
    model.eval()
    predict_tensor = model(torch.Tensor(df.loc[:, 'ep_ratio_ttm': 'BR'].values).to(device=para.device))
    probability = F.softmax(predict_tensor, dim=1).to(device='cpu').detach().numpy()[:,0]

    # 构造 factor dataframe
    factor_df = pd.DataFrame(df['date'])
    factor_df.insert(loc=0, column='probability', value=np.nan)
    factor_df['probability'] = probability

    # current_stock_id_list = factor_df.index.values.tolist()
    # current_date = factor_df['date'][0]

    factor_df = factor_df.set_index(['date', factor_df.index])
    all_factor_df = pd.concat([all_factor_df, factor_df])

    # # 构造 price dataframe
    # price_df = get_price(order_book_ids=current_stock_id_list, fields='close', start_date=current_date, end_date=current_date)
    # price_df = price_df.reset_index(level='order_book_id')
    #
    # price_df = pd.DataFrame(price_df['close'].values.tolist(), index=[price_df['order_book_id'].values.tolist()]).T
    # price_df.index = [current_date]
    # all_price_df = pd.concat([all_price_df, price_df])

c:\users\millet\miniconda3\envs\torch1.8-gpu\lib\site-packages\rqdatac\client.py:197: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)


In [14]:
all_factor_df

probability
date       order_book_id             
2017-01-03 002080.XSHE       0.460354
           600643.XSHG       0.490303
           600997.XSHG       0.484568
           600662.XSHG       0.544074
           603606.XSHG       0.450851
...                               ...
2021-11-11 002641.XSHE       0.614994
           688228.XSHG       0.544916
           002371.XSHE       0.545406
           605108.XSHG       0.546571
           002505.XSHE       0.455947

[190820 rows x 1 columns]

In [15]:
from rqdatac import all_instruments

all_price_df = get_price(order_book_ids=all_instruments(type='CS', date='2022-01-01')['order_book_id'].tolist(), fields='close', start_date='2017-01-01', end_date='2022-01-01')

all_price_df = all_price_df.unstack(level=0)

all_price_df.columns = [all_price_df.columns.tolist()[i][1] for i in range(len(all_price_df.columns.tolist()))]

QuotaExceeded: Quota exceeded

In [ ]:
all_price_df

In [ ]:
date_info_list = list()

for i_csv in range(para.month_num):
    date_info_list.append(pd.read_csv(para.data_path+'/'+str(i_csv)+'.csv', index_col='order_book_id')['date'][0])
date_info_list

In [ ]:
all_price_df = all_price_df.loc[all_price_df.index.isin(date_info_list), :]
all_price_df

In [ ]:
from alphalens.utils import get_clean_factor_and_forward_returns

a = get_clean_factor_and_forward_returns(all_factor_df['probability'], all_price_df, periods=(20, ))

In [ ]:
from alphalens import performance
from alphalens import plotting
from alphalens import tears
from alphalens import utils

In [ ]:
tears.create_returns_tear_sheet(a)